In [9]:
import random
from OCC.Display.WebGl.x3dom_renderer import *
from IPython.display import display, HTML
%matplotlib inline
caller_id = 0
def DisplayShape(shape,
                     vertex_shader=None,
                     fragment_shader=None,
                     export_edges=False,
                     color=(random.random(), random.random(), random.random()),
                     specular_color=(1, 1, 1),
                     shininess=0.9,
                     transparency=0.,
                     line_color=(0, 0., 0.),
                     line_width=2.,
                     mesh_quality=1.):
        def Show(src="<shape><appearance><material diffuseColor='0.603 0.894 0.909'></material></appearance> <box></box></shape>   ", height=400,  width=400):
            width=str(width)
            height=str(height)
            result = ""
            if(caller_id <2):
                result += " <script type='text/javascript' src='http://www.x3dom.org/download/x3dom.js'> </script>  <link rel='stylesheet' type='text/css' href='http://www.x3dom.org/download/x3dom.css'></link>"
           
            result +="<div style='height: "+height+"px;width: 100%;' id='x3dholder_"+str(caller_id)+"'  width='100%' height='"+height+"px'><x3d style='height: "+height+"px;width: 100%;' id='x3d"+str(caller_id)+"' width='100%' height='"+height+"px'><scene>"+src+"   </scene></x3d> </div>"
            return result
        x3d_exporter = X3DExporter(shape, vertex_shader, fragment_shader,
                                   export_edges, color,
                                   specular_color, shininess, transparency,
                                   line_color, line_width, mesh_quality)
        x3d_exporter.compute()
        tmp = x3d_exporter.to_x3dfile_string()
        temp_file_name = "tmp_"+str(++caller_id)+".x3d"
        if os.path.exists(temp_file_name): os.remove(temp_file_name)
        text_file = open(temp_file_name, "w")
        text_file.write(tmp)
        text_file.close()
        return HTML(Show("<inline url='./"+temp_file_name+"'> </inline> "))
    

import numpy
print(numpy.__path__)

['/opt/conda/lib/python3.5/site-packages/numpy']


In [72]:
import os
from OCC.StlAPI import StlAPI_Writer
from OCC.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.BRepPrimAPI import BRepPrimAPI_MakeCylinder
from OCC.BRepMesh import BRepMesh_IncrementalMesh
from OCC.gp import gp_Pnt, gp_Ax2, gp_Dir, gp_Circ
from OCC.GeomAPI import GeomAPI_PointsToBSpline
from OCC.TColgp import TColgp_Array1OfPnt
from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.BRepOffsetAPI import BRepOffsetAPI_MakePipe
from OCC.BRepAlgoAPI import BRepAlgoAPI_Fuse
from OCC.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.BRepAlgoAPI import BRepAlgoAPI_Cut

# first, create the shape
def pipe(point1, point2, radius):
    makeWire = BRepBuilderAPI_MakeWire()
    edge = BRepBuilderAPI_MakeEdge(point1, point2).Edge()
    makeWire.Add(edge)
    makeWire.Build()
    wire = makeWire.Wire()

    dir = gp_Dir(point2.X() - point1.X(), point2.Y() - point1.Y(), point2.Z() - point1.Z())
    circle = gp_Circ(gp_Ax2(point1,dir), radius)
    profile_edge = BRepBuilderAPI_MakeEdge(circle).Edge()
    profile_wire = BRepBuilderAPI_MakeWire(profile_edge).Wire()
    profile_face = BRepBuilderAPI_MakeFace(profile_wire).Face()
    pipe = BRepOffsetAPI_MakePipe(wire, profile_face).Shape()
    return(pipe)
    

def sphere(centre, radius):
    sphere = BRepPrimAPI_MakeSphere (centre, radius).Shape()
    return(sphere)

def pipe_3(radius_pipe, radius_sphere):
    pipe1=pipe(gp_Pnt(0,0,0), gp_Pnt(0,0,1), radius_pipe)
    sphere1=sphere(gp_Pnt(0,0,1), radius_sphere)
    pipe2=pipe(gp_Pnt(0,0,1), gp_Pnt(0,1,2), radius_pipe)  
    sphere2=sphere(gp_Pnt(0,1,2), radius_sphere)
    pipe3=pipe(gp_Pnt(0,1,2), gp_Pnt(0,2,2), radius_pipe)
    glued1 = BRepAlgoAPI_Fuse(pipe1, sphere1).Shape()
    glued2= BRepAlgoAPI_Fuse(glued1, pipe2).Shape()
    glued3 = BRepAlgoAPI_Fuse(glued2, sphere2).Shape()
    glued4 = BRepAlgoAPI_Fuse(glued3, pipe3).Shape()
    return glued4


pipe1=pipe_3(0.6, 0.6)
pipe_minus=pipe_3(0.5, 0.5)
glued4 = BRepAlgoAPI_Cut(pipe1, pipe_minus).Shape()

geometry = DisplayShape(glued4)
display(geometry)

# then mesh it. This mesh is used by the STL exporter
# Note : if the mesh is not performed, the STL exporter
# won't do nothing
mesh = BRepMesh_IncrementalMesh(glued4, 0.6)
mesh.Perform()

stl_file = "./pipe_low_resolution.stl"
stl_exporter = StlAPI_Writer()
stl_exporter.SetASCIIMode(True)  # change to False if you need binary export
stl_exporter.Write(glued4, stl_file)


# then we change the mesh resolution
#mesh.SetDeflection(0.05)


from OCC.Display.SimpleGui import init_display
from OCC.TopoDS import TopoDS_Shape
from OCC.StlAPI import StlAPI_Reader

stl_reader = StlAPI_Reader()
fan_shp = TopoDS_Shape()
stl_reader.Read(fan_shp, stl_file)

exproted = DisplayShape(fan_shp)
display(exproted)

In [73]:
import subprocess
geoCounter = 0
def getGeo(stl_filename):
    geoFile = "Merge \""+stl_filename+"\";\nSurface Loop(1) = {1};\nVolume(1) = {1};\nPhysical Volume(1) = {1};\n"
    temp_file_name = "tmp_"+str(++geoCounter)+".geo"
    if os.path.exists(temp_file_name): os.remove(temp_file_name)
    text_file = open(temp_file_name, "w")
    text_file.write(geoFile)
    text_file.close()
    return temp_file_name;
out = ""
try:
    out = subprocess.check_output(
            ["gmsh", "gmsh -3 -algo meshadapt tmp_0.geo -o SFM.msh"],
            stderr=subprocess.STDOUT
            ).strip().decode('utf8')
except subprocess.CalledProcessError as e:
    out = e.output
    
getGeo(stl_file)
print(out)

b"Unexpected end of /proc/mounts line `overlay / overlay rw,relatime,lowerdir=/var/lib/docker/overlay2/l/EZAB2MO6YA4OTNQ4VKX4ANS7NF:/var/lib/docker/overlay2/l/2ORZ3QMPOEMJNZVDU5MFHUKMSH:/var/lib/docker/overlay2/l/A2QZL7N6JSLT2XPSHVAU7GFVTD:/var/lib/docker/overlay2/l/EQII37E4TYWV4TPSZO6WFUK2XM:/var/lib/docker/overlay2/l/NIYP7JZ6RAGI34LRCE3JWNVBVJ:/var/lib/docker/overlay2/l/HDT25AD5P2NRKNIHZEU2YDTY2A:/var/lib/docker/overlay2/l/X7JV7WJJCB3WIW4NXXKJ7URRVA:/var/lib/docker/overlay2/l/EADWRVZ5MHQSYMGUWESTYGJLHR:/var/lib/docker/overlay2/l/OPHUCTPIKIK4Q'\nUnexpected end of /proc/mounts line `YLT46CLPIUGCK:/var/lib/docker/overlay2/l/NPR5JUC4FTXKMVJNCIQS56XXKI:/var/lib/docker/overlay2/l/4KUXNWD3ISHHCOESB3LO5CELOW:/var/lib/docker/overlay2/l/HTOPGBJYN7DOP65IAXZJODKSSG:/var/lib/docker/overlay2/l/HDTHWLBG4K47EXTZFNYP6MEUJG:/var/lib/docker/overlay2/l/ARHUBR5M4YADVFYKIDAH5KU5SS:/var/lib/docker/overlay2/l/ICH72TKRCDES3GJR5TSQ7OT76M:/var/lib/docker/overlay2/l/MQGLZZY3MLSYAUA3KNHWP2K72A,upperdir=/var/lib/

In [74]:
!gmsh -3 -algo meshadapt tmp_0.geo -o SFM.msh
!dolfin-convert SFM.msh sfm.xml
!ls -la

Unexpected end of /proc/mounts line `overlay / overlay rw,relatime,lowerdir=/var/lib/docker/overlay2/l/EZAB2MO6YA4OTNQ4VKX4ANS7NF:/var/lib/docker/overlay2/l/2ORZ3QMPOEMJNZVDU5MFHUKMSH:/var/lib/docker/overlay2/l/A2QZL7N6JSLT2XPSHVAU7GFVTD:/var/lib/docker/overlay2/l/EQII37E4TYWV4TPSZO6WFUK2XM:/var/lib/docker/overlay2/l/NIYP7JZ6RAGI34LRCE3JWNVBVJ:/var/lib/docker/overlay2/l/HDT25AD5P2NRKNIHZEU2YDTY2A:/var/lib/docker/overlay2/l/X7JV7WJJCB3WIW4NXXKJ7URRVA:/var/lib/docker/overlay2/l/EADWRVZ5MHQSYMGUWESTYGJLHR:/var/lib/docker/overlay2/l/OPHUCTPIKIK4Q'
Unexpected end of /proc/mounts line `YLT46CLPIUGCK:/var/lib/docker/overlay2/l/NPR5JUC4FTXKMVJNCIQS56XXKI:/var/lib/docker/overlay2/l/4KUXNWD3ISHHCOESB3LO5CELOW:/var/lib/docker/overlay2/l/HTOPGBJYN7DOP65IAXZJODKSSG:/var/lib/docker/overlay2/l/HDTHWLBG4K47EXTZFNYP6MEUJG:/var/lib/docker/overlay2/l/ARHUBR5M4YADVFYKIDAH5KU5SS:/var/lib/docker/overlay2/l/ICH72TKRCDES3GJR5TSQ7OT76M:/var/lib/docker/overlay2/l/MQGLZZY3MLSYAUA3KNHWP2K72A,upperdir=/var/lib/doc

In [75]:
from dolfin import *
mesh = Mesh("./sfm.xml")
V = FunctionSpace(mesh, "CG", 1)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0.1)
a = dot(grad(u), grad(v))*dx
L = f*v*dx

# Define boundary condition values
u0 = Constant(0.0)
u1 = Constant(1.0)
u2 = Constant(2.0)
u3 = Constant(3.0)

# Define boundary conditions
bc0 = DirichletBC(V, u0, 0)
bc1 = DirichletBC(V, u1, 1)
bc2 = DirichletBC(V, u2, 2)
bc3 = DirichletBC(V, u3, 3)

# Set PETSc MUMPS paramter (this is required to prevent a memory error
# in some cases when using MUMPS LU solver).
if has_petsc():
    PETScOptions.set("mat_mumps_icntl_14", 40.0)

# Compute solution
u = Function(V)
solve(a == L, u, [bc0, bc1, bc2, bc3])

# Write solution to file
File("u.pvd") << u
HTML(X3DOM().html(u))